In [1]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install praw
import praw

In [3]:
user_agent = "Scraper 1.0 by UnderstandingFew623"
reddit = praw.Reddit(
    client_id = "pwzg5gDB_2pTCrQ2uRpVjA",
    client_secret = "71Q8otzpNyRZMFJPRGSjtR5e6ovhbQ",
    user_agent=user_agent
  )

In [4]:
midsize = ["SEZL",
"IDCC",
"AGYS",
"BOX",
"ZETA",
"YOU",
"PAYO",
"PAY",
"LRN",
"UPWK",
"CLSK",
"CIFR",
"WHD",
"AESI",
"POST",
"AROC",
"LPX",
"LNTH",
"LMAT",
"CPRX",
"CORT",
"SEM",
"ANF",
"WRBY",
"OLLI",
"ELF",
"BROS",
"FSV",
"TRNO",
"CIGI",
"MRP",
"ABM"]


In [32]:
headlines = []

for submission in reddit.subreddit("stocks").hot(limit=400):
    upper = submission.title.upper()
    mention = [ticker for ticker in midsize if ticker in upper]

    if mention:
            headlines.append({
            "id":submission.id,
            "title": submission.title,
            "company mentioned": mention,
            "flair":submission.link_flair_text,
            "body paragraph": submission.selftext})
            

    
print(len(headlines))


38


In [33]:
sto = pd.DataFrame(headlines)
sto.head()

,id,title,company mentioned,flair,body paragraph
0,1k5recn,At what net income (from investing) do you ret...,[YOU],None,What would be the minimum that your investment...
1,1k4m0wz,What do you think is going through Elons mind ...,[YOU],None,What do you think is going through Elons mind ...
2,1k5ozeh,What do you use to invest?,[YOU],Advice Request,Do you utilize brokerages or are you fans of m...
3,1k5he0f,Warner Bros. Discovery starts Max password sha...,[BROS],None,Warner Bros. Discovery is taking a page out of...
4,1k4r1x0,"If Trump posted, ""It's a great time to buy!"" -...","[YOU, POST]",None,"It's a Tuesday morning, you are minding your o..."


In [7]:
sto.to_csv('headlines.csv', header=False, encoding='utf-8', index=False)

In [8]:

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /srv/conda/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [35]:
#vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia  =SIA()
results = []
for line in headlines:
    pol_score= sia.polarity_scores(line["title"])
    pol_score["title"] = line
    pol_score["id"] = line["id"]
    pol_score["company mentioned"] = line["company mentioned"]
    pol_score["flair"] = line["flair"]
    pol_score["body paragraph"] = line["body paragraph"]
    results.append(pol_score)

pprint(results[:3], width=100)

[{'body paragraph': 'What would be the minimum that your investments (after taxes) need to yield '
                    'for you to allow yourself to retire? That you can feel comfortable with doing '
                    'so.\n'
                    'Consider it wont be stable income. \n'
                    '\n'
                    'Some years more or less cause its still invested in high risk but managed.\n'
                    'Worst case you lose like 2% a year and thats rare, usually you just wont make '
                    'much.\n'
                    '\n'
                    '*you dont have to if you like your job but the question remains.\n'
                    'Thank you.',
  'company mentioned': ['YOU'],
  'compound': 0.0,
  'flair': None,
  'id': '1k5recn',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'title': {'body paragraph': 'What would be the minimum that your investments (after taxes) need '
                              'to yield for you to allow yourself to retire? Tha

In [10]:
stock = pd.DataFrame.from_records(results)
stock.head()

,neg,neu,pos,compound,title,id,company mentioned
0,0.0,1.000,0.000,0.0000,"{'id': '1k5recn', 'title': 'At what net income...",1k5recn,[YOU]
1,0.0,1.000,0.000,0.0000,"{'id': '1k4m0wz', 'title': 'What do you think ...",1k4m0wz,[YOU]
2,0.0,1.000,0.000,0.0000,"{'id': '1k5ozeh', 'title': 'What do you use to...",1k5ozeh,[YOU]
3,0.0,0.714,0.286,0.4215,"{'id': '1k5he0f', 'title': 'Warner Bros. Disco...",1k5he0f,[BROS]
4,0.0,0.715,0.285,0.6588,"{'id': '1k4r1x0', 'title': 'If Trump posted, ""...",1k4r1x0,"[YOU, POST]"


In [11]:
stock["label"] = 0
stock.loc[stock['compound']> 0.2, 'label'] = 1
stock.loc[stock['compound']< 0.2, 'label'] = -1
stock.head()

,neg,neu,pos,compound,title,id,company mentioned,label
0,0.0,1.000,0.000,0.0000,"{'id': '1k5recn', 'title': 'At what net income...",1k5recn,[YOU],-1
1,0.0,1.000,0.000,0.0000,"{'id': '1k4m0wz', 'title': 'What do you think ...",1k4m0wz,[YOU],-1
2,0.0,1.000,0.000,0.0000,"{'id': '1k5ozeh', 'title': 'What do you use to...",1k5ozeh,[YOU],-1
3,0.0,0.714,0.286,0.4215,"{'id': '1k5he0f', 'title': 'Warner Bros. Disco...",1k5he0f,[BROS],1
4,0.0,0.715,0.285,0.6588,"{'id': '1k4r1x0', 'title': 'If Trump posted, ""...",1k4r1x0,"[YOU, POST]",1


In [12]:
stock2 = stock[['title','label']]

In [13]:
stock2.to_csv('reddit_headlines_labels.csv', encoding='utf-8', index=False)

In [14]:
stock2.head()

,title,label
0,"{'id': '1k5recn', 'title': 'At what net income...",-1
1,"{'id': '1k4m0wz', 'title': 'What do you think ...",-1
2,"{'id': '1k5ozeh', 'title': 'What do you use to...",-1
3,"{'id': '1k5he0f', 'title': 'Warner Bros. Disco...",1
4,"{'id': '1k4r1x0', 'title': 'If Trump posted, ""...",1


In [15]:
for submission in reddit.subreddit("stocks").hot(limit=None):

    headlines.append({
        "id":submission.id,
        "title": submission.title,
        
    })

In [16]:
stock.label.value_counts()

label
-1    28
 1    10
Name: count, dtype: int64

In [17]:
stock.label.value_counts(normalize=True) *100

label
-1    73.684211
 1    26.315789
Name: proportion, dtype: float64

In [18]:
#scraping

In [19]:
stock

,neg,neu,pos,compound,title,id,company mentioned,label
0,0.000,1.000,0.000,0.0000,"{'id': '1k5recn', 'title': 'At what net income...",1k5recn,[YOU],-1
1,0.000,1.000,0.000,0.0000,"{'id': '1k4m0wz', 'title': 'What do you think ...",1k4m0wz,[YOU],-1
2,0.000,1.000,0.000,0.0000,"{'id': '1k5ozeh', 'title': 'What do you use to...",1k5ozeh,[YOU],-1
3,0.000,0.714,0.286,0.4215,"{'id': '1k5he0f', 'title': 'Warner Bros. Disco...",1k5he0f,[BROS],1
4,0.000,0.715,0.285,0.6588,"{'id': '1k4r1x0', 'title': 'If Trump posted, ""...",1k4r1x0,"[YOU, POST]",1
5,0.000,1.000,0.000,0.0000,"{'id': '1k5qhkj', 'title': 'Are you still hold...",1k5qhkj,[YOU],-1
6,0.000,1.000,0.000,0.0000,"{'id': '1k4mco3', 'title': 'How Far Do You Thi...",1k4mco3,[YOU],-1
7,0.278,0.722,0.000,-0.4019,"{'id': '1k44z2r', 'title': 'Only Trade What Yo...",1k44z2r,[YOU],-1
8,0.000,1.000,0.000,0.0000,"{'id': '1k44x1q', 'title': 'The illusion of di...",1k44x1q,[YOU],-1
9,0.000,1.000,0.000,0.0000,"{'id': '1k4bu9n', 'title': 'Options traders, w...",1k4bu9n,[YOU],-1


In [20]:
#roberta Pretrained model
!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [21]:
!pip3 install torch torchvision

MODEL = f'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [22]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores=output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = { 
        'roberta_neg': scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [23]:
from tqdm import tqdm
res = {}
for i,row in tqdm(stock.iterrows(), total=len(stock)):
    try:
        text = row["title"]
        if isinstance(text, dict):
            text = text.get("title") or text.get("text") or str(text)
        if not isinstance(text, str):
            print(f"Skipping row {i}, bad headline: {text}")
            continue
        myid = row["id"]
        vader_result = sia.polarity_scores(text)
        vader_result_rename = {}
        for key, value in vader_result.items():
            vader_result_rename[f'vader_{key}']=value
        roberta_result = polarity_scores_roberta(text)
        both = {"title":text, **vader_result_rename, **roberta_result}
        res[myid] = both
    except RuntimeError:
        print(f'Broke for id {myid}')
    

100%|██████████| 38/38 [00:22<00:00,  1.68it/s]


In [24]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index':'id'})
results_df = results_df.merge(stock, how='left', on='id')
results_df = results_df.drop(columns='title_y')
results_df.head()

,id,title_x,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,neg,neu,pos,compound,company mentioned,label
0,1k5recn,At what net income (from investing) do you ret...,0.0,1.0,0.0,0.0,0.10026,0.868474,0.031266,0.0,1.000,0.000,0.0000,[YOU],-1
1,1k4m0wz,What do you think is going through Elons mind ...,0.0,1.0,0.0,0.0,0.033995,0.930515,0.03549,0.0,1.000,0.000,0.0000,[YOU],-1
2,1k5ozeh,What do you use to invest?,0.0,1.0,0.0,0.0,0.114171,0.84324,0.042589,0.0,1.000,0.000,0.0000,[YOU],-1
3,1k5he0f,Warner Bros. Discovery starts Max password sha...,0.0,0.714,0.286,0.4215,0.138725,0.818785,0.042489,0.0,0.714,0.286,0.4215,[BROS],1
4,1k4r1x0,"If Trump posted, ""It's a great time to buy!"" -...",0.0,0.715,0.285,0.6588,0.090335,0.439,0.470665,0.0,0.715,0.285,0.6588,"[YOU, POST]",1


In [25]:
results_df["title_x"][0]

'At what net income (from investing) do you retire?'

In [26]:
results_df["title"] = results_df["title_x"].str.findall(r'[e]\'\:\s(\'[A-Za-z0-9\s\W_]+\')')
results_df = results_df.drop(columns="title_x")
